In [1]:
#!python -m pip install mysql-connector-python

In [2]:
# TODOs
# Add in metadata ability to properties
# If a duplicate property is added, allow based on given metadata

In [1]:
import pandas as pd
from time import time
from manage_compound_db import DatabaseManager
import pickle

# Ignore chaining warnings from pandas
pd.options.mode.chained_assignment = None

In [2]:
## Connect to DB ##
dbm = DatabaseManager()

# Populate DB

In [3]:
dbm.open_db()
# dbm.run_query('DELETE FROM property;', no_return=True)
# dbm.run_query('DELETE FROM category;', no_return=True)
dbm.run_query('INSERT IGNORE INTO category (name) values (\'Name\'), (\'SMILES\'), (\'InChI\'), (\'Formula\')', no_return=True)
dbm.conn.commit()
dbm.close_db()

In [4]:
# Populate with some cpds from Dorrenstein's cpds
t = time()

rename_dict = {'RDKit_InchIKey':'inchi_key', 'NCGC_ID':'lib_id', '[M+H] CCS (DarkChem v1.0)':'[M+H] CCS', 
               '[M+Na] CCS (DarkChem v1.0)':'[M+Na] CCS', '[M-H] CCS (DarkChem v1.0)':'[M-H] CCS', 
               'RDKit_Canon_SMILES':'SMILES'}
dorr = dbm.prep_input('dorrenstein.xlsx', rename_dict=rename_dict)
dbm.populate_db(dorr, 'Dorrenstein')

print('Time taken: %.1f min' % ((time() - t) / 60))

dorr.head()

Library: 1 entries added successfully. 0 not added.

Compound: 4221 entries added successfully. 165 not added.

LibraryEntry: 4303 entries added successfully. 83 not added.

Mass: 4221 entries added successfully. 165 not added.

ccs_mph: 3244 entries added successfully. 143 not added.

ccs_mpna: 3244 entries added successfully. 143 not added.

ccs_mmh: 3244 entries added successfully. 143 not added.

The following columns could not be added to properties table: Conc (mM), index, [M+Na] CCS, lib_id, XLogP, OriginalSMILES, Mass, Notes, [M-H] CCS, Well, [M+H] CCS, inchi_key, Volume, Position
Property: 10399 entries added successfully. 453 not added.

Time taken: 0.2 min


,index,Position,Well,lib_id,Conc (mM),Volume,OriginalSMILES,SMILES,Formula,Mass,Name,XLogP,inchi_key,Notes,[M+H] CCS,[M+Na] CCS,[M-H] CCS,cpd_id
0,0,GnPS_01_A2,A2,NCGC00380516-01,NaN,NaN,CC(=O)OCC(CCC=C(C)C)=CCCC(CO)=CCCC(C)=CCO,CC(=O)OCC(=CCCC(=CCCC(C)=CCO)CO)CCC=C(C)C,C22H36O4,364.261360,[12-hydroxy-6-(hydroxymethyl)-10-methyl-2-(4-m...,4.7,GFDOSVZSTRYXMK-UHFFFAOYSA-N,NaN,192.19077,194.04282,187.02477,0
1,1,GnPS_01_A2,A2,NCGC00015878-06,NaN,NaN,COC(=O)C1C(O)CCC2CN3CCC4=C(NC5=C4C=CC=C5)C3CC12,COC(=O)C1C(O)CCC2CN3CCc4c([nH]c5ccccc45)C3CC21,C21H26N2O3,354.194343,"methyl 18-hydroxy-1,3,11,12,14,15,16,17,18,19,...",2.9,BLGXFZZNTVWLAY-UHFFFAOYSA-N,NaN,183.73230,187.89206,180.24869,1
2,2,GnPS_01_A2,A2,NCGC00015878-06,NaN,NaN,COC(=O)C1C(O)CCC2CN3CCC4=C(NC5=C4C=CC=C5)C3CC12,COC(=O)C1C(O)CCC2CN3CCc4c([nH]c5ccccc45)C3CC21,C21H26N2O3,354.194343,"methyl 18-hydroxy-1,3,11,12,14,15,16,17,18,19,...",2.9,BLGXFZZNTVWLAY-UHFFFAOYSA-N,NaN,183.73230,187.89206,180.24869,1
3,3,GnPS_01_A2,A2,NCGC00380523-01,NaN,NaN,COC1=C2C=CC(=O)OC2=C(OCC(OC(=O)C(\C)=C\C)C(C)(...,C/C=C(\C)C(=O)OC(COc1c2occc2c(OC)c2ccc(=O)oc12...,C22H24O8,416.147118,"[3-hydroxy-1-(4-methoxy-7-oxofuro[3,2-g]chrome...",3.3,QUSMURJIBTWAOP-WUXMJOGZSA-N,NaN,195.59264,198.60199,189.22792,2
4,4,GnPS_01_A2,A2,NCGC00180826-02,NaN,NaN,C\C=C(\C)C(=O)OC1C(OC(C)=O)C2=C3OC(=O)C=CC3=CC...,C/C=C(/C)C(=O)OC1C(OC(C)=O)c2c(ccc3ccc(=O)oc23...,C21H22O7,386.136553,"(10-acetyloxy-8,8-dimethyl-2-oxo-9,10-dihydrop...",3.1,XGPBRZDOJDLKOT-WDZFZDKYSA-N,NaN,190.11950,191.31277,185.35875,3


In [5]:
# Populate with HMDB
t = time()

rename_dict = {'RDKit_InChIKey':'inchi_key', 'LibID':'lib_id', 'MetCCS [M+H] CCS':'[M+H] CCS', 
               'MetCCS [M+Na] CCS':'[M+Na] CCS', 'MetCCS [M-H] CCS':'[M-H] CCS',
              'RDKit_Canon_SMILES':'SMILES'}
# hmdb = dbm.prep_input('hmdb.xlsx', rename_dict=rename_dict)
# with open('hmdb.pkl', 'wb') as f:
#     pickle.dump(hmdb, f)
with open('hmdb.pkl', 'rb') as f:
    hmdb = pickle.load(f)
dbm.populate_db(hmdb, 'HMDB')

print('Time taken: %.1f min' % ((time() - t) / 60))

hmdb.head()

Library: 1 entries added successfully. 0 not added.

Compound: 113672 entries added successfully. 541 not added.

LibraryEntry: 114213 entries added successfully. 0 not added.

Mass: 113672 entries added successfully. 541 not added.

ccs_mph: 34186 entries added successfully. 397 not added.

ccs_mpna: 34186 entries added successfully. 397 not added.

ccs_mmh: 34186 entries added successfully. 397 not added.

Here
Here
Here
Here
Here
Here
Here
Here
Here
MS2_Spectra: 75312 entries added successfully. 0 not added.

Fragment_mph: 963611 entries added successfully. 11394 not added.

Here
Here
Here
Here
Here
Here
Here
Here
Here
Here
MS2_Spectra: 150536 entries added successfully. 0 not added.

Fragment_mph: 1973318 entries added successfully. 20768 not added.

Here
Here
Here
Here
Here
Here
Here
Here
Here
Here
Here
MS2_Spectra: 225760 entries added successfully. 0 not added.

Fragment_mph: 3166001 entries added successfully. 25193 not added.

Here
Here
Here
Here
Here
Here
Here
Here
MS2_Spectr

,index,lib_id,InChI,InChIKey,OriginalSMILES,SMILES,inchi_key,Name,Direct Parent,Kingdom Class,...,[M+H] CCS,[M+Na] CCS,[M-H] CCS,MSMS [M+H] 10,MSMS [M+H] 20,MSMS [M+H] 40,MSMS [M-H] 10,MSMS [M-H] 20,MSMS [M-H] 40,cpd_id
0,0,HMDB0000001,InChI=1S/C7H11N3O2/c1-10-3-5(9-4-10)2-6(8)7(11...,BRMWTNUJHUMWMS-LURJTMIESA-N,CN1C=NC(C[C@H](N)C(O)=O)=C1,Cn1cnc(C[C@H](N)C(=O)O)c1,BRMWTNUJHUMWMS-LURJTMIESA-N,(2S)-2-amino-3-(1-methyl-1H-imidazol-4-yl)prop...,Histidine and derivatives,Chemical entities,...,138.8,142.7,135.5,"18.0344,0.0380;28.0187,0.1201;30.0344,0.0457;4...","28.0187,0.3031;42.0344,0.1269;56.0136,0.1591;6...","28.0187,3.4780;41.0391,1.1694;42.0344,3.0969;4...","16.0187,0.0379;17.0027,0.1596;26.0031,0.2075;2...","26.0031,0.5818;28.0187,1.2426;44.0500,0.6925;4...","26.0031,2.4779;28.0187,3.3267;41.0140,0.8966;4...",4221
1,1,HMDB0000002,InChI=1S/C3H10N2/c4-2-1-3-5/h1-5H2,XFNJVJPLKCPIBV-UHFFFAOYSA-N,NCCCN,NCCCN,XFNJVJPLKCPIBV-UHFFFAOYSA-N,"propane-1,3-diamine",Monoalkylamines,Organic compounds,...,105.5,110.6,113.5,"18.0344,0.1524;27.0235,0.1086;29.0391,0.0503;3...","18.0344,0.0350;27.0235,0.2549;29.0391,0.1086;3...","18.0344,0.4473;27.0235,5.2901;29.0391,1.2191;3...","15.0235,0.0024;16.0187,1.5250;25.0078,0.0016;2...","15.0235,0.0216;16.0187,1.8915;25.0078,0.0110;2...","15.0235,0.0871;16.0187,7.2422;25.0078,0.6294;2...",4222
2,2,HMDB0000005,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,CCC(=O)C(O)=O,CCC(=O)C(=O)O,TYEYBOSBBBHJIV-UHFFFAOYSA-N,2-oxobutanoic acid,Short-chain keto acids and derivatives,Organic compounds,...,130.8,138.1,112.7,"19.0184,0.0041;27.0235,0.4264;29.0391,1.1981;3...","19.0184,0.1599;27.0235,0.5626;29.0391,8.5557;3...","19.0184,0.0098;27.0235,9.3757;29.0391,10.8706;...","17.0027,0.1505;27.0235,0.0357;29.0027,0.0842;3...","17.0027,0.2682;27.0235,0.2235;29.0027,1.0791;3...","17.0027,0.1336;27.0235,1.2984;29.0027,3.2627;3...",4223
3,3,HMDB0000008,"InChI=1S/C4H8O3/c1-2-3(5)4(6)7/h3,5H,2H2,1H3,(...",AFENDNXGAFYKQO-UHFFFAOYSA-N,CCC(O)C(O)=O,CCC(O)C(=O)O,AFENDNXGAFYKQO-UHFFFAOYSA-N,2-hydroxybutanoic acid,Alpha hydroxy acids and derivatives,Organic compounds,...,126.9,133.8,113.4,"19.0184,0.1044;27.0235,0.3233;29.0391,0.5770;3...","19.0184,0.1148;27.0235,0.8266;29.0391,11.5795;...","19.0184,0.2844;27.0235,4.2757;29.0391,6.7086;3...","17.0027,0.1724;27.0235,0.0502;29.0027,0.5471;2...","17.0027,0.3940;27.0235,0.3046;29.0027,1.0533;2...","17.0027,0.2500;27.0235,1.6486;29.0027,5.5182;2...",4224
4,4,HMDB0000010,InChI=1S/C19H24O3/c1-19-8-7-12-13(15(19)5-6-18...,WHEUWNKSCXYKBU-QPWUGHHJSA-N,[H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[...,COc1cc2c(cc1O)CC[C@@H]1[C@@H]2CC[C@]2(C)C(=O)C...,WHEUWNKSCXYKBU-QPWUGHHJSA-N,"(1S,10R,11S,15S)-5-hydroxy-4-methoxy-15-methyl...",Estrogens and derivatives,Chemical entities,...,175.2,182.7,176.6,"111.0804,1.0421;123.0441,1.0366;137.0597,1.622...","39.0229,1.1029;109.0648,1.1052;111.0804,1.1009...","39.0229,3.0470;51.0229,3.8822;53.0386,2.0651;5...","257.1547,0.5250;267.1391,0.4972;271.1704,0.616...","267.1391,2.5481;269.1547,2.8673;271.1704,4.593...","41.0033,10.4959;55.0189,1.8543;95.0502,1.4214;...",4225


# View a duplicate

In [3]:
# Compound
dbm.run_query('SELECT * FROM compound WHERE cpd_id = 96')

,cpd_id,inchi_key
0,96,BOLQJTPHPSDZHR-UHFFFAOYSA-N


In [4]:
# Library
dbm.run_query('SELECT * FROM library')

,name,doi,last_update
0,Dorrenstein,None,None
1,HMDB,None,None


In [5]:
# Category
dbm.run_query('SELECT * FROM category')

,name
0,Formula
1,InChI
2,Name
3,SMILES


In [6]:
# Library entries
dbm.run_query('SELECT * FROM libraryentry WHERE cpd_id = 96')

,lib_id,lib_name,cpd_id
0,HMDB0062121,HMDB,96
1,NCGC00380184-01,Dorrenstein,96


In [6]:
dorr[dorr['cpd_id'] == 96][['lib_id', 'SMILES', 'Formula', 'Mass', 'Name', 'inchi_key', '']

,index,Position,Well,lib_id,Conc (mM),Volume,OriginalSMILES,SMILES,Formula,Mass,Name,XLogP,inchi_key,Notes,[M+H] CCS,[M+Na] CCS,[M-H] CCS,cpd_id
102,102,GnPS_02_C6,C6,NCGC00380184-01,NaN,NaN,COC1=C(O)C=CC(CCC(O)=O)=C1,COc1cc(CCC(=O)O)ccc1O,C10H12O4,196.073559,3-(4-hydroxy-3-methoxyphenyl)propanoic acid,0.7,BOLQJTPHPSDZHR-UHFFFAOYSA-N,NaN,145.5789,154.78593,144.0764,96


In [7]:
hmdb[hmdb['cpd_id'] == 96]

,index,lib_id,InChI,InChIKey,OriginalSMILES,SMILES,inchi_key,Name,Direct Parent,Kingdom Class,...,[M+H] CCS,[M+Na] CCS,[M-H] CCS,MSMS Positive 10,MSMS Positive 20,MSMS Positive 40,MSMS Negative 10,MSMS Negative 20,MSMS Negative 40,cpd_id
41992,42013,HMDB0062121,InChI=1S/C10H12O4/c1-14-9-6-7(2-4-8(9)11)3-5-1...,BOLQJTPHPSDZHR-UHFFFAOYSA-N,COC1=C(O)C=CC(CCC(O)=O)=C1,COc1cc(CCC(=O)O)ccc1O,BOLQJTPHPSDZHR-UHFFFAOYSA-N,3-(4-hydroxy-3-methoxyphenyl)propanoic acid,Phenylpropanoic acids,Chemical entities,...,NaN,NaN,NaN,"27.0235,0.0954;43.0184,0.2261;44.9977,0.0968;5...","27.0235,0.5181;43.0184,0.8308;53.0391,0.2842;5...","27.0235,4.4569;41.0391,1.5537;43.0184,1.7109;4...","17.0027,0.2289;31.0184,0.2484;41.0027,0.1523;4...","17.0027,0.2885;41.0027,0.7402;44.9977,1.0770;5...","41.0027,10.0166;44.9977,5.4573;55.0184,2.4289;...",96


In [10]:
# Properties available
# Note HMDB loaded second, and it has InChIs & MS2, but no CCS, unlike Dorrenstein's lib
dbm.run_query('SELECT * FROM property WHERE cpd_id = 96')

,cpd_id,category,value,property_id
0,96,Formula,C10H12O4,594713
1,96,InChI,InChI=1S/C10H12O4/c1-14-9-6-7(2-4-8(9)11)3-5-1...,648619
2,96,Name,3-(4-hydroxy-3-methoxyphenyl)propanoic acid,598930
3,96,SMILES,COc1cc(CCC(=O)O)ccc1O,590492


In [7]:
dbm.run_query('SELECT * FROM ms2_spectra WHERE cpd_id = 96')

,cpd_id,adduct,voltage,spectra_id
0,96,mph,10,39750
1,96,mph,20,115001
2,96,mph,40,190225
3,96,mmh,10,262191
4,96,mmh,20,331494
5,96,mmh,40,400797


In [10]:
dbm.run_query('SELECT * FROM fragment_mph WHERE spectra_id = 39750')

,spectra_id,mass,relative_intensity
0,39750,27.0235,0.0010
1,39750,43.0184,0.0023
2,39750,44.9977,0.0010
3,39750,55.0184,0.0043
4,39750,61.0290,0.0034
5,39750,71.0133,0.0036
6,39750,73.0290,0.0092
7,39750,85.0290,0.0011
8,39750,87.0446,0.0012
9,39750,95.0133,0.0009


# Compounds unique to libraries

In [6]:
# Library entries
cmd = 'SELECT * FROM libraryentry ORDER BY cpd_id'
df = dbm.run_query(cmd)
df

,lib_id,lib_name,cpd_id
0,NCGC00380516-01,Dorrenstein,0
1,NCGC00015878-06,Dorrenstein,1
2,NCGC00015878-07,Dorrenstein,1
3,NCGC00015878-08,Dorrenstein,1
4,NCGC00380523-01,Dorrenstein,2
...,...,...,...
118511,HMDB0240289,HMDB,117888
118512,HMDB0240290,HMDB,117889
118513,HMDB0240291,HMDB,117890
118514,HMDB0240292,HMDB,117891


In [7]:
# Remove compounds present in >1 lib
df = df.drop_duplicates(subset='cpd_id', keep=False)
df

,lib_id,lib_name,cpd_id
0,NCGC00380516-01,Dorrenstein,0
4,NCGC00380523-01,Dorrenstein,2
5,NCGC00180826-02,Dorrenstein,3
6,NCGC00169771-03,Dorrenstein,4
7,NCGC00347686-02,Dorrenstein,5
...,...,...,...
118511,HMDB0240289,HMDB,117888
118512,HMDB0240290,HMDB,117889
118513,HMDB0240291,HMDB,117890
118514,HMDB0240292,HMDB,117891


In [8]:
# View compounds unique to each lib
dorr_only = df[df['lib_name'] == 'Dorrenstein']
hmdb_only = df[df['lib_name'] == 'HMDB']
print('Unique to Dorrenstein:\n', dorr_only)
print('\nUnique to HMDB:\n', hmdb_only)

Unique to Dorrenstein:
                lib_id     lib_name  cpd_id
0     NCGC00380516-01  Dorrenstein       0
4     NCGC00380523-01  Dorrenstein       2
5     NCGC00180826-02  Dorrenstein       3
6     NCGC00169771-03  Dorrenstein       4
7     NCGC00347686-02  Dorrenstein       5
...               ...          ...     ...
4620  NCGC00179061-05  Dorrenstein    4215
4621  NCGC00385998-01  Dorrenstein    4216
4624  NCGC00385999-01  Dorrenstein    4218
4625  NCGC00247521-02  Dorrenstein    4219
4626  NCGC00386111-01  Dorrenstein    4220

[3831 rows x 3 columns]

Unique to HMDB:
              lib_id lib_name  cpd_id
4627    HMDB0000001     HMDB    4221
4628    HMDB0000002     HMDB    4222
4629    HMDB0000005     HMDB    4223
4630    HMDB0000008     HMDB    4224
4631    HMDB0000010     HMDB    4225
...             ...      ...     ...
118511  HMDB0240289     HMDB  117888
118512  HMDB0240290     HMDB  117889
118513  HMDB0240291     HMDB  117890
118514  HMDB0240292     HMDB  117891
118515  HM

# Create wide table of properties

In [9]:
# Gwt InChIKeys
hmdb_ikey = dbm.run_query('SELECT * FROM compound ORDER BY cpd_id')
hmdb_ikey = hmdb_only.set_index('cpd_id').join(hmdb_ikey.set_index('cpd_id'))
hmdb_ikey

,lib_id,lib_name,inchi_key
cpd_id,,,
4221,HMDB0000001,HMDB,BRMWTNUJHUMWMS-LURJTMIESA-N
4222,HMDB0000002,HMDB,XFNJVJPLKCPIBV-UHFFFAOYSA-N
4223,HMDB0000005,HMDB,TYEYBOSBBBHJIV-UHFFFAOYSA-N
4224,HMDB0000008,HMDB,AFENDNXGAFYKQO-UHFFFAOYSA-N
4225,HMDB0000010,HMDB,WHEUWNKSCXYKBU-QPWUGHHJSA-N
...,...,...,...
117888,HMDB0240289,HMDB,GFSTXYOTEVLASN-UHFFFAOYSA-N
117889,HMDB0240290,HMDB,BSRTYNDWQXVCKR-UHFFFAOYSA-N
117890,HMDB0240291,HMDB,MBBZMMPHUWSWHV-BDVNFPICSA-N


In [10]:
# Get properties
cmd = 'SELECT * FROM property ORDER BY cpd_id'
property_df = dbm.run_query(cmd)
property_df

,property_id,cpd_id,category,value
0,1,0,SMILES,CC(=O)OCC(=CCCC(=CCCC(C)=CCO)CO)CCC=C(C)C
1,4387,0,Formula,C22H36O4
2,8773,0,Name,[12-hydroxy-6-(hydroxymethyl)-10-methyl-2-(4-m...
3,10845,0,[M+H] CCS,192.19077
4,14232,0,[M+Na] CCS,194.04282
...,...,...,...,...
577416,249430,117891,SMILES,O=C(O)C(O)[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
577417,363307,117892,Name,Diphenylmercury
577418,135218,117892,InChI,InChI=1S/2C6H5.Hg/c2*1-2-4-6-5-3-1;/h2*1-5H;
577419,477520,117892,Formula,C12H10Hg


In [11]:
# Convert property table to wide format
wide_property_df = property_df.pivot(index='cpd_id', columns='category', values='value')
wide_property_df

category,Formula,InChI,Name,SMILES,[M+H] CCS,[M+Na] CCS,[M-H] CCS
cpd_id,,,,,,,
0,C22H36O4,NaN,[12-hydroxy-6-(hydroxymethyl)-10-methyl-2-(4-m...,CC(=O)OCC(=CCCC(=CCCC(C)=CCO)CO)CCC=C(C)C,192.19077,194.04282,187.02477
1,C21H26N2O3,NaN,"methyl 18-hydroxy-1,3,11,12,14,15,16,17,18,19,...",COC(=O)C1C(O)CCC2CN3CCc4c([nH]c5ccccc45)C3CC21,183.7323,187.89206,180.24869
2,C22H24O8,NaN,"[3-hydroxy-1-(4-methoxy-7-oxofuro[3,2-g]chrome...",C/C=C(C)C(=O)OC(COc1c2occc2c(OC)c2ccc(=O)oc12)...,195.59264,198.60199,189.22792
3,C21H22O7,NaN,"(10-acetyloxy-8,8-dimethyl-2-oxo-9,10-dihydrop...",C/C=C(/C)C(=O)OC1C(OC(C)=O)c2c(ccc3ccc(=O)oc23...,190.1195,191.31277,185.35875
4,C20H18O7,NaN,"(3R,3aS,6S,6aR)-3,6-bis(1,3-benzodioxol-5-yl)-...",O[C@]12CO[C@H](c3ccc4c(c3)OCO4)[C@H]1CO[C@@H]2...,185.02568,189.90298,178.37335
...,...,...,...,...,...,...,...
117888,C16H28GdN4O8,InChI=1S/C16H28N4O8.Gd/c21-13(22)9-17-1-2-18(1...,Gadoteric acid,O=C(O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O)C...,NaN,NaN,NaN
117889,(C2H2Cl2)nC2H6,"InChI=1S/C4H8Cl2/c1-3-4(2,5)6/h3H2,1-2H3",Saran,CCC(C)(Cl)Cl,NaN,NaN,NaN
117890,C7H17NO5,InChI=1S/C7H17NO5/c1-8-2-4(10)6(12)7(13)5(11)3...,Meglumine,CNC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,NaN,NaN,NaN


In [12]:
# Join with unique HMDB cpds and their InChIKeys
final_hmdb = hmdb_ikey.join(wide_property_df)
final_hmdb = final_hmdb.drop('lib_name', axis=1)
final_hmdb

,lib_id,inchi_key,Formula,InChI,Name,SMILES,[M+H] CCS,[M+Na] CCS,[M-H] CCS
cpd_id,,,,,,,,,
4221,HMDB0000001,BRMWTNUJHUMWMS-LURJTMIESA-N,C7H11N3O2,InChI=1S/C7H11N3O2/c1-10-3-5(9-4-10)2-6(8)7(11...,(2S)-2-amino-3-(1-methyl-1H-imidazol-4-yl)prop...,Cn1cnc(C[C@H](N)C(=O)O)c1,138.8,142.7,135.5
4222,HMDB0000002,XFNJVJPLKCPIBV-UHFFFAOYSA-N,C3H10N2,InChI=1S/C3H10N2/c4-2-1-3-5/h1-5H2,"propane-1,3-diamine",NCCCN,105.5,110.6,113.5
4223,HMDB0000005,TYEYBOSBBBHJIV-UHFFFAOYSA-N,C4H6O3,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",2-oxobutanoic acid,CCC(=O)C(=O)O,130.8,138.1,112.7
4224,HMDB0000008,AFENDNXGAFYKQO-UHFFFAOYSA-N,C4H8O3,"InChI=1S/C4H8O3/c1-2-3(5)4(6)7/h3,5H,2H2,1H3,(...",2-hydroxybutanoic acid,CCC(O)C(=O)O,126.9,133.8,113.4
4225,HMDB0000010,WHEUWNKSCXYKBU-QPWUGHHJSA-N,C19H24O3,InChI=1S/C19H24O3/c1-19-8-7-12-13(15(19)5-6-18...,"(1S,10R,11S,15S)-5-hydroxy-4-methoxy-15-methyl...",COc1cc2c(cc1O)CC[C@@H]1[C@@H]2CC[C@]2(C)C(=O)C...,175.2,182.7,176.6
...,...,...,...,...,...,...,...,...,...
117888,HMDB0240289,GFSTXYOTEVLASN-UHFFFAOYSA-N,C16H28GdN4O8,InChI=1S/C16H28N4O8.Gd/c21-13(22)9-17-1-2-18(1...,Gadoteric acid,O=C(O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O)C...,NaN,NaN,NaN
117889,HMDB0240290,BSRTYNDWQXVCKR-UHFFFAOYSA-N,(C2H2Cl2)nC2H6,"InChI=1S/C4H8Cl2/c1-3-4(2,5)6/h3H2,1-2H3",Saran,CCC(C)(Cl)Cl,NaN,NaN,NaN
117890,HMDB0240291,MBBZMMPHUWSWHV-BDVNFPICSA-N,C7H17NO5,InChI=1S/C7H17NO5/c1-8-2-4(10)6(12)7(13)5(11)3...,Meglumine,CNC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,NaN,NaN,NaN


# Example MAME Queries & Timings

In [11]:
# Start with example feature and settings
feature_mass = 118.0867
feature_ccs = 126
adduct = '[M+H]'
adduct_mz = 1.0078
mass_error = 5 / 1e6  # ppm
ccs_error = 3 / 1e2  # percent

time_n = 1000
time_n_slow = 10
mz = feature_mass - adduct_mz

In [14]:
# Get matches by mass
query = ('SELECT * FROM mass WHERE value BETWEEN {:.4f} and {:.4f}')
min_mass = mz * (1 - mass_error)
max_mass = mz * (1 + mass_error)
query = query.format(min_mass, max_mass)

dbm.set_stay_open(True)

t = time()
for i in range(time_n):
    mass_matches1 = dbm.run_query(query)
print('Time per query (direct): %.3f ms' % ((time() - t) / time_n * 1000))

t = time()
for i in range(time_n):
    mass_matches2 = dbm.fetch_matches_within_error('mass', 'value', mz, rel_error=mass_error, num_dec=4)
print('Time per query (indirect): %.3f ms' % ((time() - t) / time_n * 1000))

dbm.set_stay_open(False)

print('All same hits: {}'.format((mass_matches1 == mass_matches2).all(axis=None)))

mass_matches1

Time per query (direct): 40.692 ms
Time per query (indirect): 40.596 ms
All same hits: True


,cpd_id,value
0,4249,117.079
1,4911,117.079
2,5272,117.079
3,5635,117.079
4,6035,117.079
5,12732,117.079
6,13955,117.079


In [35]:
# Get matches by mass - timing for query itself when stay_open=False
query = ('SELECT cpd_id, value mass FROM mass WHERE value BETWEEN %.4f and %.4f')
min_mass = mz * (1 - mass_error)
max_mass = mz * (1 + mass_error)

dbm.set_stay_open(False)
t = time()
for i in range(time_n_slow):
    mass_matches = dbm.run_query(query % (min_mass, max_mass))
print('Time per query: %.3f ms' % ((time() - t) / time_n_slow * 1000))

mass_matches

Time per query: 94.836 ms


,cpd_id,mass
0,4249,117.079
1,4911,117.079
2,5272,117.079
3,5635,117.079
4,6035,117.079
5,12732,117.079
6,13955,117.079


In [13]:
# Get matches by MS2 fragment mass
# Start here: why doesn't this hack work?
# query = ('SELECT * FROM fragment_mph WHERE mass BETWEEN {:.2f} and {:.2f} AND mass BETWEEN {:.4f} and {:.4f}')
query = ('SELECT * FROM fragment_mph WHERE mass BETWEEN {:.2f} and {:.2f}')
frag_mz = 151.04
min_mass = frag_mz * (1 - 1e2 / 1e6)  # 100 ppm error for ms2
max_mass = frag_mz * (1 + 1e2 / 1e6)
query = query.format(min_mass, max_mass, min_mass, max_mass)

dbm.set_stay_open(True)

t = time()
for i in range(time_n_slow):
    ms2_matches1 = dbm.run_query(query)
print('Time per query (direct): %.3f ms' % ((time() - t) / time_n_slow * 1000))

# t = time()
# for i in range(2):
#     ms2_matches2 = dbm.fetch_matches_within_error('fragment', 'mass', frag_mz, rel_error=mass_error)
# print('Time per query (indirect): %.3f ms' % ((time() - t) / 2 * 1000))
dbm.set_stay_open(False)

# print('All same hits: {}'.format((ms2_matches1 == ms2_matches2).all(axis=None)))

ms2_matches1

Time per query (direct): 1982.266 ms


,spectra_id,mass,relative_intensity
0,71,151.040,0.0056
1,81,151.040,0.2964
2,121,151.036,0.0083
3,123,151.051,0.0117
4,189,151.026,0.0390
...,...,...,...
2342,225730,151.055,0.0169
2343,225742,151.055,0.0124
2344,225745,151.055,0.0150
2345,225749,151.040,0.2844


In [16]:
# Get matches by CCS
query = ('SELECT * FROM ccs_mph WHERE value BETWEEN {:.1f} and {:.1f}')
min_ccs = feature_ccs * (1 - ccs_error)
max_ccs = feature_ccs * (1 + ccs_error)
query = query.format(min_ccs, max_ccs)

dbm.set_stay_open(True)

t = time()
for i in range(time_n):
    ccs_matches1 = dbm.run_query(query)
print('Time per query (direct): %.3f ms' % ((time() - t) / time_n * 1000))

# t = time()
# for i in range(time_n):
#     ccs_matches2 = dbm.fetch_property_match('[M+H] CCS', feature_ccs, rel_error=ccs_error, num_dec=2)
# print('Time per query (indirect): %.3f ms' % ((time() - t) / time_n * 1000))

# print('All same hits: {}'.format((ccs_matches1 == ccs_matches2).all(axis=None)))

dbm.set_stay_open(False)

ccs_matches1

Time per query (direct): 22.611 ms


,cpd_id,value
0,51,129.323
1,228,122.215
2,231,125.526
3,1088,127.596
4,1457,124.637
...,...,...
1139,44826,126.300
1140,44877,124.500
1141,44897,123.500
1142,45072,125.100


In [16]:
# Refine the CCS results a bit
ccs_matches = ccs_matches1[['cpd_id', 'value']]
ccs_matches.rename(columns={'value':('%s CCS' % adduct)}, inplace=True)
ccs_matches

,cpd_id,[M+H] CCS
0,51,129.32309
1,231,125.526245
2,1088,127.59596
3,1457,124.63654
4,1499,129.04558
...,...,...
1138,44826,126.3
1139,44877,124.5
1140,44897,123.5
1141,45072,125.1


In [17]:
# Return those passing both
df1 = mass_matches[(mass_matches['cpd_id'].isin(ccs_matches['cpd_id']))]
df2 = ccs_matches[(ccs_matches['cpd_id'].isin(mass_matches['cpd_id']))]
df1.set_index('cpd_id').join(df2.set_index('cpd_id'), on='cpd_id')

,value,[M+H] CCS
cpd_id,,
4249,117.079,124.9
4911,117.079,125.5
5635,117.079,125.8
6035,117.079,125.2
12732,117.079,125.8


In [4]:
# Return those passing both - SQL side
min_mass, max_mass = dbm.get_range(mz, rel_error=mass_error)
min_ccs, max_ccs = dbm.get_range(feature_ccs, rel_error=ccs_error)

# Design query
query = '''
        SELECT mass.cpd_id, mass.value mass, ccs_mph.value ccs
        FROM mass
        LEFT JOIN ccs_mph
        ON mass.cpd_id=ccs_mph.cpd_id
        WHERE mass.value BETWEEN {:.4f} and {:.4f}
        AND ccs_mph.value BETWEEN {:.2f} and {:.2f}
        '''
query = query.format(min_mass, max_mass,
                     min_ccs, max_ccs)

dbm.set_stay_open(True)

# t = time()
# for i in range(time_n):
#     matches1 = dbm.run_query(query)
# print('Time per query (direct): %.3f ms' % ((time() - t) / time_n * 1000))

t = time()
for i in range(time_n):
    matches2 = dbm.fetch_mass_ccs_matches(mz, feature_ccs, '[M+H]', mass_rel_error=mass_error, ccs_rel_error=ccs_error)
print('Time per query (indirect): %.3f ms' % ((time() - t) / time_n * 1000))

dbm.set_stay_open(False)

# print('All same hits: {}'.format((matches1 == matches2).all(axis=None)))
matches2

Time per query (indirect): 16.577 ms


,cpd_id,mass,[M+H]_CCS
0,4249,117.079,124.9
1,4911,117.079,125.5
2,5635,117.079,125.8
3,6035,117.079,125.2
4,12732,117.079,125.8


In [5]:
t = time()
dbm.set_stay_open(True)
for i in range(time_n):
    matches2 = dbm.fetch_mass_ccs_matches(mz, feature_ccs, '[M-H]', mass_rel_error=mass_error, ccs_rel_error=ccs_error)
print('Time per query (indirect): %.3f ms' % ((time() - t) / time_n * 1000))
dbm.set_stay_open(False)

# print('All same hits: {}'.format((matches1 == matches2).all(axis=None)))
matches2

Time per query (indirect): 15.895 ms


,cpd_id,mass,[M-H]_CCS
0,5272,117.079,122.5
1,5635,117.079,122.6
2,6035,117.079,123.2
3,12732,117.079,122.8
4,13955,117.079,123.3


In [3]:
spectra = dbm.fetch_ms2_spectra(cpd_id=96, adduct='[M+H]', voltage=10)

spectra[0]

,mass,relative_intensity
0,27.0235,0.0010
1,43.0184,0.0023
2,44.9977,0.0010
3,55.0184,0.0043
4,61.0290,0.0034
5,71.0133,0.0036
6,73.0290,0.0092
7,85.0290,0.0011
8,87.0446,0.0012
9,95.0133,0.0009


In [5]:
spectra[0].name

'96, [M+H], 10, 39750'